# Benchmark study on single sample gene set analysis tools

#### Contributor: Antonio Mora, Chengshu Xie 
#### Date of first version: 2019-01-20
#### Date of last review: 2020-05-26 
#### Summary:
This is a workflow to compare 6 single sample gene set analysis methods(PLAGE, ZSCORE, SSGSEA, GSVA, GRAPE, Pathifier) through the slected real benchmark datasets. Here are the 8 data sets about respiratory disease from GEO database, which are available in [GitHub](https://github.com/mora-lab/benchmarks/tree/master/single-sample/data):

* [GSE10245 : Non-Small Cells Lung Cancer](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE10245)
* [GSE11906 : Chronic Obstructive Pulmanory Disease](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE11906)
* [GSE18842 : Non-Small Cells Lung Cancer](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE18842)
* [GSE35571 : Asthma](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE35571)
* [GSE42057 : Chronic Obstructive Pulmanory Disease](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE42057)
* [GSE50834 : Tuberculosis](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE50834)
* [GSE52819 : Tuberculosis](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE52819)
* [GSE67472 : Asthma](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE67472)

#### Contents:
* [1. Data Preparation](#link1)
    * [1.1 Prerequisites](#link2)
    
* [2. Method application](#link)
* [3. Calculation of sensitivity, specificity, and precision](#link) 
* [4. Make boxplots](#link)




## <a id=link1>1. Data Preparation</a>

### <a id=link2>1.1 Prerequisites</a>

Install and library R packages in need:

In [6]:
suppressPackageStartupMessages(library(R.utils))
suppressPackageStartupMessages(library(GEOquery))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(pathifier))
suppressPackageStartupMessages(library(GRAPE))
suppressPackageStartupMessages(library(GSVAdata))
suppressPackageStartupMessages(library(GSEABase))
suppressPackageStartupMessages(library(GSVA))
suppressPackageStartupMessages(library(limma))
suppressPackageStartupMessages(library(metap))
suppressPackageStartupMessages(library(ggplot2))

In [ ]:
get_microarray_data = function(geo_accession, pdata_file, GPL, gpl_file){  
  
  ### Download and filter microarray data
  geo_data = getGEO(geo_accession, destdir=".",getGPL = F);
  exprSet = as.data.frame(exprs(geo_data[[1]]));
  pdata = read.csv(pdata_file, header = T, row.names = 1);
  exprSet = exprSet[which(colnames(exprSet) %in% rownames(pdata))];
  exprSet = exprSet[, order(colnames(exprSet))];
  pdata = pdata[order(rownames(pdata)), ];
  
  ### Read or Download GPL file, and load it
  if (file.exists(gpl_file) == TRUE){
    gpl <- getGEO(filename=paste(GPL,".soft", sep = ""))
  } else {
    gpl <- getGEO(GPL, destdir=".")
  }
  gpl_data = Table(gpl);
  
  if (GPL == "GPL570"){
    symbol = gpl_data$`Gene Symbol`;
    gpl_data$symbol = unlist(lapply(symbol, function(x) word(x, 1, sep="\\ /// ")));  # extract symbols
    gpl_data = gpl_data[, which(colnames(gpl_data) %in% c("ID","symbol"))];
  }
  
  if (GPL == "GPL10558"){
    gpl_data = gpl_data[which(gpl_data$Species ==  "Homo sapiens"),];   # extract symbols
    gpl_data = gpl_data[, which(colnames(gpl_data) %in% c("ID","Symbol"))];
    colnames(gpl_data) = c("ID","symbol");
  }
  
  if (GPL == "GPL6244"){
    symbol = gpl_data$`gene_assignment`;
    gpl_data$symbol = unlist(lapply(symbol, function(x) word(x, 2, sep="\\ // "))); # extract symbols
    gpl_data = gpl_data[, which(colnames(gpl_data) %in% c("ID","symbol"))];
  }  
  
  if (GPL == "GPL16311"){
    # library(biomaRt)
    Symbols <- mget(as.character(na.omit(gpl_data$SPOT_ID)), org.Hs.egSYMBOL, ifnotfound=NA); # convert entrez id into symbols 
    Symbols = as.data.frame(unlist(Symbols));
    colnames(Symbols) = "symbol";
    Symbols$SPOT_ID = rownames(Symbols);
    gpl_data$SPOT_ID = as.character(gpl_data$SPOT_ID);
    gpl_data = na.omit(Symbols) %>%
      inner_join(.,gpl_data, by = "SPOT_ID");
    gpl_data = gpl_data[, which(colnames(gpl_data) %in% c("ID","symbol"))]
  }  
  
  new_exprSet = exprSet
  new_exprSet$ID =  as.character(rownames(new_exprSet));
  
  #### convert probe ids into 
  newdata = new_exprSet %>% 
    merge(.,gpl_data, by="ID") %>% #merge informatio of probe_id
    dplyr::select(-ID) %>% #remove extra column    
    dplyr::select(symbol, everything()) %>% #re-arrange
    mutate(rowMean = rowMeans(.[grep("GSM", names(.))])) %>% #calculate means
    arrange(desc(rowMean))  %>% #order the means
    distinct(symbol,.keep_all = T) %>% # keep the first symbol information
    dplyr::select(-rowMean) #  #remove the new colmun
  newdata = as.data.frame(newdata);
  newdata = newdata[which(newdata$symbol != ""),]
  rownames(newdata) = newdata$symbol ;
  newdata = dplyr::select(newdata, -"symbol") # remove extra column
  newdata = rbind(pdata$Normal,newdata);
  rownames(newdata)[1] = "normal" ; 
  newdata = na.omit(newdata);
  newdata
}

In [ ]:
GSE10245 = get_microarray_data(geo_accession = "GSE10245", 
                               pdata_file = "E:/SSbenchmark_test/data/GSE10245_pdata.csv",
                               GPL = "GPL570",
                               gpl_file = "E:/SSbenchmark_test/GPL570.soft");
GSE11906 = get_microarray_data(geo_accession = "GSE11906",
                               pdata_file = "E:/SSbenchmark_test/data/GSE11906_pdata.csv",
                               GPL = "GPL570",
                               gpl_file = "E:/SSbenchmark_test/GPL570.soft");
GSE18842 = get_microarray_data(geo_accession = "GSE18842",
                               pdata_file = "E:/SSbenchmark_test/data/GSE18842_pdata.csv",
                               GPL = "GPL570",
                               gpl_file = "E:/SSbenchmark_test/GPL570.soft");
GSE35571 = get_microarray_data(geo_accession = "GSE35571", 
                               pdata_file = "E:/SSbenchmark_test/data/GSE35571_pdata.csv",
                               GPL = "GPL570",
                               gpl_file = "E:/SSbenchmark_test/GPL570.soft");
GSE42057 = get_microarray_data(geo_accession = "GSE42057", 
                               pdata_file = "E:/SSbenchmark_test/data/GSE42057_pdata.csv",
                               GPL = "GPL570",
                               gpl_file = "E:/SSbenchmark_test/GPL570.soft");
GSE50834 = get_microarray_data(geo_accession = "GSE50834",
                               pdata_file = "E:/SSbenchmark_test/data/GSE50834_pdata.csv",
                               GPL = "GPL10558",
                               gpl_file = "E:/SSbenchmark_test/GPL10558.soft");
GSE52819 = get_microarray_data(geo_accession = "GSE52819",
                               pdata_file = "E:/SSbenchmark_test/data/GSE52819_pdata.csv",
                               GPL = "GPL6244",
                               gpl_file = "E:/SSbenchmark_test/GPL6244.soft");
GSE67472 = get_microarray_data(geo_accession = "GSE67472",
                               pdata_file = "E:/SSbenchmark_test/data/GSE67472_pdata.csv",
                               GPL = "GPL16311",
                               gpl_file = "E:/SSbenchmark_test/GPL16311.soft");


GSEdata = list("GSE10245" = GSE10245, "GSE11906" = GSE11906,
               "GSE18842" = GSE18842, "GSE35571" = GSE35571,
               "GSE42057" = GSE42057, "GSE50834" = GSE50834,
               "GSE52819" = GSE52819, "GSE67472" = GSE67472);

In [1]:
format_input_pathifier = function(GSEdata){
  
  # Prepare data and parameters 
  # Extract information from binary phenotypes. 1 = Control, 0 = Disease
  normals = as.vector(as.logical(GSEdata[1,]));
  exp_matrix = as.matrix(GSEdata[-1,]);
  
  # samples information
  samples = as.list(colnames(exp_matrix));
  # Calculate MIN_STD
  New_exp_matrix = exp_matrix[,as.logical(normals)];
  nsd = apply(New_exp_matrix, 1, sd);
  min_std = quantile(nsd, 0.25);
  
  # Calculate MIN_EXP
  min_exp = quantile(as.vector(exp_matrix), 0.1); # Percentile 10 of data
  
  # Filter low value genes. At least 10% of samples with values over min_exp
  # Set expression levels < MIN_EXP to MIN_EXP
  greater = apply(exp_matrix, 1, function(x) x > min_exp);
  new_greater = apply(greater, 2, mean);
  new_greater = names(new_greater)[new_greater > 0.1];
  exp_matrix = exp_matrix[new_greater,];
  exp_matrix[exp_matrix < min_exp] = min_exp;
  
  # Set maximum 5000 genes with more variance
  # V = names(sort(apply(exp_matrix, 1, var), decreasing = T))#[1:10000]
  # V = V[!is.na(V)]
  # exp_matrix = exp_matrix[V,]
  allgenes = as.vector(rownames(exp_matrix));
  pathifier_input <- list();
  pathifier_input$data <- exp_matrix;
  pathifier_input$samples <- samples;
  pathifier_input$normals <- normals;
  pathifier_input$min_std <- min_std;
  pathifier_input$min_exp <- min_exp;
  pathifier_input$allgenes <- allgenes;
  pathifier_input
}


Loading required package: limma



In [ ]:
pathifier_input = lapply(GSEdata, format_input_pathifier);

In [6]:
read_gmt = function(file){
  if(!grepl("\\.gmt$",file)[1]){stop("Pathway information must be a .gmt file")}  
  geneSet = readLines(file)                                     ##read in the gmt file as a vector of lines
  geneSet = strsplit(geneSet,"\t")                            ##convert from vector of strings to a list
  names(geneSet) = sapply(geneSet,"[",1)                      ##move the names column as the names of the list
  geneSet = lapply(geneSet, "[",-1:-2)                        ##remove name and description columns
  geneSet = lapply(geneSet, function(x){x[which(x!="")]})     ##remove empty strings
  
  ### Sort the pathway, pathway with more genes comes first.
  geneSet.tmp = lengths(geneSet)
  geneSet.sort = sort(geneSet.tmp, decreasing = TRUE)
  newGS = list()
  for (i in 1:length(geneSet)){
    newGS[i] = geneSet[names(geneSet.sort)[i]]
    names(newGS)[i] = names(geneSet.sort)[i]
  }    
  return(newGS)
  ## pdata = read.csv(pdata_file, header = T, row.names = 1);
  ## genesetcollection = getGmt("E:/SSbenchmark_test/data/c2.cp.kegg.v7.0.symbols.gmt", 
  ## geneIdType = EntrezIdentifier(), 
  ## collectionType = BroadCollection(category="c2"));
  
}


### function to get pathway data to be used 
# source("E:/SSbenchmark_test//R/read_gmt.R")
pathwaylist = read_gmt("E:/SSbenchmark_test/data/c2.cp.kegg.v7.0.symbols.gmt");
target.Asthma = read_gmt("E:/SSbenchmark_test/data/Asthma.target.pathway.symbols.gmt");
target.COPD = read_gmt("E:/SSbenchmark_test/data/COPD.target.pathway.symbols.gmt");
target.NSCLC = read_gmt("E:/SSbenchmark_test/data/NSCLC.target.pathway.symbols.gmt");
target.Tuber = read_gmt("E:/SSbenchmark_test/data/Tuberclosis.target.pathway.symbols.gmt");
genesetcollection = getGmt("E:/SSbenchmark_test/data/c2.cp.kegg.v7.0.symbols.gmt", 
                           geneIdType = EntrezIdentifier(), 
                           collectionType = BroadCollection(category="c2"));	

In [ ]:
run_gsva_4methods = function(expdata, genesetcollection){
  result.plage = list();
  result.zscore = list();
  result.ssgsea = list();
  result.gsva = list();
  result.plage = gsva(as.matrix(expdata[-1,]), genesetcollection, method = "plage", 
                      mx.diff = FALSE, parallel.sz=2, abs.ranking = FALSE, verbose=TRUE)
  result.zscore = gsva(as.matrix(expdata[-1,]), genesetcollection, method = "zscore", 
                       mx.diff = FALSE, parallel.sz=2, abs.ranking = FALSE, verbose=TRUE)
  result.ssgsea = gsva(as.matrix(expdata[-1,]), genesetcollection, method = "ssgsea",
                       mx.diff = FALSE, parallel.sz=2, abs.ranking = FALSE, verbose=TRUE)
  result.gsva = gsva(as.matrix(expdata[-1,]), genesetcollection, method = "gsva",
                     mx.diff = FALSE, parallel.sz=2, abs.ranking = FALSE, verbose=TRUE)
  result = list("result.plage" = result.plage, "result.zscore" = result.zscore,
                "result.ssgsea" = result.ssgsea, "result.gsva" = result.gsva)
  result
}

### results 
gsva_4results = lapply(GSEdata, function(x) run_gsva_4methods(x, genesetcollection))

adjust_str = function(resultlist){
  
  score.result = list();
  i = 1;
  for (j in 1:4){
    score.result[[j]] = list("GSE10245" = gsva_4results[[i]][[j]],
                             "GSE11906" = gsva_4results[[i+1]][[j]],
                             "GSE18842" = gsva_4results[[i+2]][[j]],
                             "GSE35571" = gsva_4results[[i+3]][[j]],
                             "GSE42057" = gsva_4results[[i+4]][[j]],
                             "GSE50834" = gsva_4results[[i+5]][[j]],
                             "GSE52819" = gsva_4results[[i+6]][[j]],
                             "GSE67472" = gsva_4results[[i+7]][[j]])
  }
  names(score.result) = c("result.plage", 
                          "result.zscore",
                          "result.ssgsea", 
                          "result.gsva")
  return(score.result)
}

newscore.result = adjust_str(gsva_4results)

In [ ]:
PATHWAYS = list();
PATHWAYS$gs = pathwaylist;
PATHWAYS$pathwaynames = as.list(names(pathwaylist));
names(PATHWAYS$pathwaynames) = names(pathwaylist);

### function to get pathway data to be used 
run_pathifier = function(new_gsedata, PATHWAYS){
  result = quantify_pathways_deregulation(new_gsedata$data,
                                          new_gsedata$allgenes,
                                          PATHWAYS$gs,
                                          PATHWAYS$pathwaynames,
                                          new_gsedata$normals,
                                          # maximize_stability = T,
                                          attempts = 100,
                                          min_std = new_gsedata$min_std,
                                          min_exp = new_gsedata$min_exp);
  
  result.PDS = t(mapply(FUN = c, result$scores));
  # colnames(result.PDS) = colnames(gsedata$data)
  result.PDS
}

### results
pathifier_result = lapply(pathifier_input, function(x) run_pathifier(x, PATHWAYS));


In [ ]:
run_GRAPE = function(expData, PathwayList){
  
  ### divide the data into two parts
  ConData = expData[-1,][ , which(expData[1,] == 1)] 
  ConData = as.matrix(log(ConData)) 
  
  
  psmat = makeGRAPE_psMat(ConData, log(expData[-1,]), PathwayList) 
  colnames(psmat) = colnames(expData[-1,]) 
  psmat
  
}

result.GRAPE = lapply(GSEdata, function(x) run_GRAPE(x, pathwaylist))

result.GRAPE_GSE10245 = run_GRAPE(GSE10245, pathwaylist);
result.GRAPE_GSE11906 = run_GRAPE(GSE11906, pathwaylist);
result.GRAPE_GSE18842 = run_GRAPE(GSE18842, pathwaylist);
result.GRAPE_GSE35571 = run_GRAPE(GSE35571, pathwaylist);
result.GRAPE_GSE50834 = run_GRAPE(GSE50834, pathwaylist);
result.GRAPE_GSE52819 = run_GRAPE(GSE52819, pathwaylist);
result.GRAPE_GSE67472 = run_GRAPE(GSE67472, pathwaylist);

## not work
result.GRAPE_GSE42057 = run_GRAPE(GSE42057, pathwaylist);

GRAPE_result = list("GSE10245" = result.GRAPE_GSE10245,
                    "GSE11906" = result.GRAPE_GSE11906,
                    "GSE18842" = result.GRAPE_GSE18842,
                    "GSE35571" = result.GRAPE_GSE35571,
                    # "GSE42057" = matrix(),
                    "GSE50834" = result.GRAPE_GSE50834,
                    "GSE52819" = result.GRAPE_GSE52819,
                    "GSE67472" = result.GRAPE_GSE67472)


In [ ]:
SSP_calculation = function(P.res.data, target.pathway){
  
  cond_below_0.05 = c();
  cond_over_0.05 = c();
  
  true_positives = list();
  true_positives_data = c();
  false_negatives = c();
  false_negatives_data = c();
  true_negatives_ids = list();
  false_positives1_ids = list();
  false_positives2_ids = list();
  false_positives = list();
  
  greater_than_0.05 = c();
  less_than_0.05 = c();
  true_negatives_ids = c();
  false_positives = c();
  sensitivity = c();
  specificity = c();
  precision = c();
  
  sensitivity_result = c();
  specificity_result = c();
  precision_result = c();
  
  for (i in 1:ncol(P.res.data)) {
    
    cond_below_0.05[[i]] = rownames(P.res.data[which(P.res.data[,i] <= 0.05),]) %in% toupper(names(target.pathway))
    cond_over_0.05[[i]] = rownames(P.res.data[which(P.res.data[,i] > 0.05),]) %in% toupper(names(target.pathway))
    
    true_positives_data  = P.res.data[which(cond_below_0.05[[i]]),]
    if(class(true_positives_data) == "matrix"){ true_positives = as.numeric(length(true_positives_data[,i])) }
    false_negatives_data  = P.res.data[which(cond_over_0.05[[i]]),]
    if(class(false_negatives_data) == "matrix"){ false_negatives = as.numeric(length(false_negatives_data[,i])) }
    if(class(false_negatives_data) == "numeric"){ false_negatives = 1 }
    
    ## Tool results' subsets on the basis of statistical significance.
    greater_than_0.05[[i]] = as.data.frame(P.res.data[which(P.res.data[,i] > 0.05), i]);
    less_than_0.05[[i]] = as.data.frame(P.res.data[which(P.res.data[,i] <= 0.05),i]);
    true_negatives_ids[[i]] = setdiff(rownames(greater_than_0.05[[i]]), names(target.pathway)); ## All ids that are there in the tool result with p > 0.05 and absent in the disease pool.
    
    false_positives1_ids = intersect(rownames(greater_than_0.05[[i]]), names(target.pathway));
    false_positives2_ids = setdiff(rownames(less_than_0.05[[i]]), names(target.pathway));
    false_positives[[i]] = c(false_positives1_ids,false_positives2_ids);
    
    sensitivity[[i]] = true_positives/(true_positives+false_negatives);
    specificity[[i]] = length(true_negatives_ids[[i]])/(length(true_negatives_ids[[i]])+length(false_positives[[i]]));
    precision[[i]] = true_positives/(true_positives+length(false_positives[[i]]))
  }
  
  sensitivity.re = data.frame(t(sapply(sensitivity,c)))
  specificity.re = data.frame(t(sapply(specificity,c)))
  precision.re = data.frame(t(sapply(precision,c)))
  colnames(sensitivity.re) = colnames(specificity.re) = colnames(precision.re) = colnames(P.res.data)
  
  result = list("sensitivity.result" = sensitivity.re, 
                "specificity.result" = specificity.re,
                "precision.result" = precision.re)
  return(result)
}
GSE10245_NSCLC = SSP_calculation(P.res[[1]], target.NSCLC)
GSE18842_NSCLC = SSP_calculation(P.res[[3]], target.NSCLC)
GSE11906_COPD = SSP_calculation(P.res[[2]], target.COPD)
GSE35571_Asthma = SSP_calculation(P.res[[4]], target.Asthma)
##################################################################
### boxplot plot
library(ggplot2)
library(plotROC)  ### insatll.packages()

sensitivity_result = read.csv("E:/04 COPD_GSE36221/sensitivity_results1219.csv", header = T)
ROC_result = read.csv("E:/04 COPD_GSE36221/ROC_results1219.csv", header = T)
#### boxplot 1
senplot <- ggplot(sensitivity_result, aes(Method,Sensitivity, fill = Method))+
  geom_boxplot()+
  xlab("Sensitivity")+
  ylab("P-values")
speplot <- ggplot(sensitivity_result, aes(Method,Specificity, fill = Method))+
  geom_boxplot()+
  xlab("Specificity")+
  ylab("P-values")
prelot <- ggplot(sensitivity_result, aes(Method,Precision, fill = Method))+
  geom_boxplot() +
  scale_y_sqrt()+
  xlab("Precision")+
  ylab("P-values")

In [ ]:
sessionInfo()